# RNA-seq Analysis Pipeline: *Picosynechococcus sp.* PCC 11901

**Project**: Transcriptomic analysis of PCC 11901 under various nutrient, environmental, and circadian conditions

**Experimental Design**:
- 20 conditions × 3 biological replicates = 60 samples
- Group 1: Nutrient conditions (Control: U4,5,6)
- Group 2: Environmental conditions (Control: U1,2,3)
- Group 3: Circadian rhythm (T1-T4 timepoints)

**Pipeline Overview**:
1. Quality Control (FastQC + MultiQC)
2. Trimming (fastp)
3. Lane Merging
4. rRNA Removal (SortMeRNA)
5. Quantification (Salmon)
6. Alignment for Visualization (Bowtie2 → bigWig)
7. Differential Expression (DESeq2 via rpy2)
8. Visualization

---

## 0. Setup and Configuration

In [ ]:
# === Standard Library ===
import os
import subprocess
import glob
import re
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# === Fix for rpy2 on Apple Silicon ===
# Force R_HOME to conda env's R before importing rpy2
# This prevents conflicts with System R
if 'CONDA_PREFIX' in os.environ:
    os.environ['R_HOME'] = os.path.join(os.environ['CONDA_PREFIX'], 'lib', 'R')

# === Data Science ===
import numpy as np
import pandas as pd
from scipy import stats

# === Visualization ===
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === Progress & Parallelization ===
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

# === R Integration ===
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

# Activate pandas/R dataframe conversion
pandas2ri.activate()

# Set plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
pd.set_option('display.max_columns', 50)

print("All packages imported successfully!")

In [ ]:
# === Project Configuration ===

# Base directory (adjust if needed)
BASE_DIR = Path(os.getcwd())

# Directory structure
DIRS = {
    'data': BASE_DIR / 'Data',
    'genome': BASE_DIR / 'PCC_11901_annotated genome',
    'qc': BASE_DIR / '01_QC',
    'trimmed': BASE_DIR / '02_trimmed',
    'merged': BASE_DIR / '03_merged',
    'rrna_filtered': BASE_DIR / '04_rRNA_filtered',
    'salmon': BASE_DIR / '05_salmon',
    'alignment': BASE_DIR / '06_alignment',
    'deseq2': BASE_DIR / '07_deseq2',
    'functional': BASE_DIR / '08_functional',
    'figures': BASE_DIR / '09_figures',
    'logs': BASE_DIR / 'logs',
}

# Reference files
GENOME_FASTA = DIRS['genome'] / 'GCF_005577135.1_ASM557713v1_genomic.fna'
GTF_FILE = DIRS['genome'] / 'genomic.gtf'
TRANSCRIPTOME_FASTA = DIRS['genome'] / 'transcriptome.fa'  # Will be generated

# Hardware settings (M2 Max optimized)
N_THREADS = 8  # General parallelization
SORTMERNA_THREADS = 4  # Memory-intensive (~15GB/thread)
SALMON_THREADS = 10

# Create output directories
for name, path in DIRS.items():
    if name not in ['data', 'genome']:  # Don't create input dirs
        path.mkdir(parents=True, exist_ok=True)
        
# Create subdirectories
(DIRS['qc'] / 'fastqc_raw').mkdir(exist_ok=True)
(DIRS['qc'] / 'fastqc_trimmed').mkdir(exist_ok=True)
(DIRS['qc'] / 'multiqc_reports').mkdir(exist_ok=True)
(DIRS['rrna_filtered'] / 'non_rRNA').mkdir(exist_ok=True)
(DIRS['rrna_filtered'] / 'rRNA').mkdir(exist_ok=True)
(DIRS['salmon'] / 'index').mkdir(exist_ok=True)
(DIRS['salmon'] / 'quants').mkdir(exist_ok=True)
(DIRS['alignment'] / 'bowtie2_index').mkdir(exist_ok=True)
(DIRS['alignment'] / 'bam').mkdir(exist_ok=True)
(DIRS['alignment'] / 'bigwig').mkdir(exist_ok=True)
(DIRS['figures'] / 'qc_plots').mkdir(exist_ok=True)
(DIRS['figures'] / 'volcano_plots').mkdir(exist_ok=True)
(DIRS['figures'] / 'heatmaps').mkdir(exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Output directories created.")

In [ ]:
# === Sample Metadata ===

# Define experimental design
sample_info = {
    # Group 1 - Nutrients (Control: U4,5,6)
    'U4': {'group': 1, 'condition': 'Control_MAD', 'replicate': 1},
    'U5': {'group': 1, 'condition': 'Control_MAD', 'replicate': 2},
    'U6': {'group': 1, 'condition': 'Control_MAD', 'replicate': 3},
    'U46': {'group': 1, 'condition': 'Glycerol_0.75pct', 'replicate': 1},
    'U47': {'group': 1, 'condition': 'Glycerol_0.75pct', 'replicate': 2},
    'U48': {'group': 1, 'condition': 'Glycerol_0.75pct', 'replicate': 3},
    'U22': {'group': 1, 'condition': 'Low_Nitrogen', 'replicate': 1},
    'U23': {'group': 1, 'condition': 'Low_Nitrogen', 'replicate': 2},
    'U24': {'group': 1, 'condition': 'Low_Nitrogen', 'replicate': 3},
    'U25': {'group': 1, 'condition': 'High_Nitrogen', 'replicate': 1},
    'U26': {'group': 1, 'condition': 'High_Nitrogen', 'replicate': 2},
    'U27': {'group': 1, 'condition': 'High_Nitrogen', 'replicate': 3},
    'U28': {'group': 1, 'condition': 'Low_Phosphate', 'replicate': 1},
    'U29': {'group': 1, 'condition': 'Low_Phosphate', 'replicate': 2},
    'U30': {'group': 1, 'condition': 'Low_Phosphate', 'replicate': 3},
    'U31': {'group': 1, 'condition': 'High_Phosphate', 'replicate': 1},
    'U32': {'group': 1, 'condition': 'High_Phosphate', 'replicate': 2},
    'U33': {'group': 1, 'condition': 'High_Phosphate', 'replicate': 3},
    'U40': {'group': 1, 'condition': 'Ammonia', 'replicate': 1},
    'U41': {'group': 1, 'condition': 'Ammonia', 'replicate': 2},
    'U42': {'group': 1, 'condition': 'Ammonia', 'replicate': 3},
    'U43': {'group': 1, 'condition': 'Urea', 'replicate': 1},
    'U44': {'group': 1, 'condition': 'Urea', 'replicate': 2},
    'U45': {'group': 1, 'condition': 'Urea', 'replicate': 3},
    
    # Group 2 - Environmental (Control: U1,2,3)
    'U1': {'group': 2, 'condition': 'Control_MAD', 'replicate': 1},
    'U2': {'group': 2, 'condition': 'Control_MAD', 'replicate': 2},
    'U3': {'group': 2, 'condition': 'Control_MAD', 'replicate': 3},
    'U34': {'group': 2, 'condition': 'High_NaCl_9pct', 'replicate': 1},
    'U35': {'group': 2, 'condition': 'High_NaCl_9pct', 'replicate': 2},
    'U36': {'group': 2, 'condition': 'High_NaCl_9pct', 'replicate': 3},
    'U37': {'group': 2, 'condition': 'H2O2_0.005pct', 'replicate': 1},
    'U38': {'group': 2, 'condition': 'H2O2_0.005pct', 'replicate': 2},
    'U39': {'group': 2, 'condition': 'H2O2_0.005pct', 'replicate': 3},
    'U7': {'group': 2, 'condition': 'Atmospheric_CO2', 'replicate': 1},
    'U8': {'group': 2, 'condition': 'Atmospheric_CO2', 'replicate': 2},
    'U9': {'group': 2, 'condition': 'Atmospheric_CO2', 'replicate': 3},
    'U10': {'group': 2, 'condition': 'High_CO2_8pct', 'replicate': 1},
    'U11': {'group': 2, 'condition': 'High_CO2_8pct', 'replicate': 2},
    'U12': {'group': 2, 'condition': 'High_CO2_8pct', 'replicate': 3},
    'U13': {'group': 2, 'condition': 'High_Temp_38C', 'replicate': 1},
    'U14': {'group': 2, 'condition': 'High_Temp_38C', 'replicate': 2},
    'U15': {'group': 2, 'condition': 'High_Temp_38C', 'replicate': 3},
    'U16': {'group': 2, 'condition': 'Low_Light_15uE', 'replicate': 1},
    'U17': {'group': 2, 'condition': 'Low_Light_15uE', 'replicate': 2},
    'U18': {'group': 2, 'condition': 'Low_Light_15uE', 'replicate': 3},
    'U19': {'group': 2, 'condition': 'High_Light', 'replicate': 1},
    'U20': {'group': 2, 'condition': 'High_Light', 'replicate': 2},
    'U21': {'group': 2, 'condition': 'High_Light', 'replicate': 3},
    
    # Group 3 - Circadian
    'U49': {'group': 3, 'condition': 'T1_Light', 'replicate': 1},
    'U50': {'group': 3, 'condition': 'T1_Light', 'replicate': 2},
    'U51': {'group': 3, 'condition': 'T1_Light', 'replicate': 3},
    'U52': {'group': 3, 'condition': 'T2_Dark', 'replicate': 1},
    'U53': {'group': 3, 'condition': 'T2_Dark', 'replicate': 2},
    'U54': {'group': 3, 'condition': 'T2_Dark', 'replicate': 3},
    'U55': {'group': 3, 'condition': 'T3_Light', 'replicate': 1},
    'U56': {'group': 3, 'condition': 'T3_Light', 'replicate': 2},
    'U57': {'group': 3, 'condition': 'T3_Light', 'replicate': 3},
    'U58': {'group': 3, 'condition': 'T4_Dark', 'replicate': 1},
    'U59': {'group': 3, 'condition': 'T4_Dark', 'replicate': 2},
    'U60': {'group': 3, 'condition': 'T4_Dark', 'replicate': 3},
}

# Create DataFrame
metadata = pd.DataFrame.from_dict(sample_info, orient='index')
metadata.index.name = 'sample_id'
metadata = metadata.reset_index()

# Sort by sample number
metadata['sample_num'] = metadata['sample_id'].str.extract(r'U(\d+)').astype(int)
metadata = metadata.sort_values('sample_num').reset_index(drop=True)

print(f"Total samples: {len(metadata)}")
print(f"\nGroup summary:")
print(metadata.groupby('group')['condition'].nunique())
metadata.head(10)

In [ ]:
# === Helper Functions ===

def run_command(cmd, description="Running command", log_file=None, check=True):
    """
    Run a shell command with optional logging.
    
    Parameters:
    -----------
    cmd : str
        Shell command to execute
    description : str
        Description for progress display
    log_file : Path, optional
        File to write stdout/stderr
    check : bool
        Raise exception on non-zero exit code
    
    Returns:
    --------
    subprocess.CompletedProcess
    """
    print(f"  {description}...")
    
    if log_file:
        with open(log_file, 'w') as f:
            result = subprocess.run(
                cmd, shell=True, stdout=f, stderr=subprocess.STDOUT,
                check=check
            )
    else:
        result = subprocess.run(
            cmd, shell=True, capture_output=True, text=True, check=check
        )
    
    return result


def get_fastq_files(sample_id, directory, pattern='merged'):
    """
    Get FASTQ file paths for a sample.
    
    Parameters:
    -----------
    sample_id : str
        Sample ID (e.g., 'U1')
    directory : Path
        Directory containing FASTQ files
    pattern : str
        'raw' for lane-split files, 'merged' for merged files
    
    Returns:
    --------
    dict with 'R1' and 'R2' keys
    """
    if pattern == 'merged':
        r1 = directory / f"{sample_id}_R1.fastq.gz"
        r2 = directory / f"{sample_id}_R2.fastq.gz"
        return {'R1': r1, 'R2': r2}
    else:
        # Raw files (4 per sample)
        files = list(directory.glob(f"{sample_id}-*.fastq.gz"))
        r1 = sorted([f for f in files if '_R1_' in f.name])
        r2 = sorted([f for f in files if '_R2_' in f.name])
        return {'R1': r1, 'R2': r2}


def check_tool(tool_name):
    """Check if a tool is available in PATH."""
    result = subprocess.run(f"which {tool_name}", shell=True, capture_output=True)
    if result.returncode == 0:
        print(f"  [OK] {tool_name}: {result.stdout.decode().strip()}")
        return True
    else:
        print(f"  [MISSING] {tool_name}")
        return False


print("Helper functions defined.")

In [ ]:
# === Verify Tool Installation ===

print("Checking required tools...\n")

tools = [
    'fastqc', 'multiqc', 'fastp',  # QC
    'sortmerna',                     # rRNA removal
    'gffread',                       # Transcriptome extraction
    'salmon',                        # Quantification
    'bowtie2', 'samtools',           # Alignment
    'bamCoverage',                   # deepTools
]

missing = []
for tool in tools:
    if not check_tool(tool):
        missing.append(tool)

if missing:
    print(f"\n[WARNING] Missing tools: {missing}")
    print("Install with: conda activate pcc11901_rnaseq")
else:
    print("\nAll tools available!")

---
## Phase 1: Pre-processing

### 1.1 Quality Control of Raw Reads

In [ ]:
# === FastQC on Raw Reads ===
# 
# Shell command equivalent:
# fastqc -t 8 -o 01_QC/fastqc_raw/ Data/*.fastq.gz
#
# This step checks:
# - Per-base sequence quality
# - Per-sequence quality scores
# - GC content distribution
# - Sequence duplication levels
# - Overrepresented sequences (adapters)
# - Adapter content

RUN_FASTQC_RAW = False  # Set to True to run

if RUN_FASTQC_RAW:
    output_dir = DIRS['qc'] / 'fastqc_raw'
    input_files = list(DIRS['data'].glob('*.fastq.gz'))
    
    print(f"Running FastQC on {len(input_files)} files...")
    print(f"Output: {output_dir}")
    
    cmd = f"fastqc -t {N_THREADS} -o {output_dir} {DIRS['data']}/*.fastq.gz"
    print(f"\nCommand:\n{cmd}\n")
    
    # Run FastQC
    !{cmd}
    
    print("\nFastQC complete!")
else:
    print("Skipping FastQC on raw reads (set RUN_FASTQC_RAW = True to run)")
    print(f"\nTo run manually:\nfastqc -t {N_THREADS} -o {DIRS['qc']}/fastqc_raw/ {DIRS['data']}/*.fastq.gz")

### 1.2 Trimming with fastp

In [ ]:
# === Trimming with fastp ===
#
# fastp automatically:
# - Detects and removes adapters
# - Trims low-quality bases from both ends
# - Filters reads below quality threshold
# - Generates HTML/JSON QC reports
#
# For each sample, we process the 4 raw FASTQ files (2 lanes × 2 reads)
# and output trimmed files ready for lane merging.

RUN_FASTP = False  # Set to True to run

def run_fastp_sample(sample_id):
    """
    Run fastp on a single sample (all lanes).
    """
    raw_files = get_fastq_files(sample_id, DIRS['data'], pattern='raw')
    
    # Process each lane separately
    for lane in ['L001', 'L002']:
        r1_in = [f for f in raw_files['R1'] if lane in f.name][0]
        r2_in = [f for f in raw_files['R2'] if lane in f.name][0]
        
        r1_out = DIRS['trimmed'] / f"{sample_id}_{lane}_R1.fastq.gz"
        r2_out = DIRS['trimmed'] / f"{sample_id}_{lane}_R2.fastq.gz"
        
        json_report = DIRS['trimmed'] / f"{sample_id}_{lane}.fastp.json"
        html_report = DIRS['trimmed'] / f"{sample_id}_{lane}.fastp.html"
        
        cmd = f"""
fastp \\
    -i "{r1_in}" \\
    -I "{r2_in}" \\
    -o "{r1_out}" \\
    -O "{r2_out}" \\
    --detect_adapter_for_pe \\
    --correction \\
    --qualified_quality_phred 20 \\
    --length_required 50 \\
    --thread {N_THREADS} \\
    --json "{json_report}" \\
    --html "{html_report}"
"""
        subprocess.run(cmd, shell=True, check=True, capture_output=True)
    
    return sample_id


if RUN_FASTP:
    samples = metadata['sample_id'].tolist()
    print(f"Running fastp on {len(samples)} samples...")
    print(f"Output: {DIRS['trimmed']}")
    
    # Run in parallel (but fastp itself uses threads, so limit parallel jobs)
    results = Parallel(n_jobs=2)(
        delayed(run_fastp_sample)(s) for s in tqdm(samples, desc="Trimming")
    )
    
    print(f"\nCompleted trimming for {len(results)} samples.")
else:
    print("Skipping fastp trimming (set RUN_FASTP = True to run)")
    print("\nExample command for single sample:")
    print(f"""
fastp \\
    -i Data/U1-AMO17076A1-22FLL5LT1_S1_L001_R1_001.fastq.gz \\
    -I Data/U1-AMO17076A1-22FLL5LT1_S1_L001_R2_001.fastq.gz \\
    -o 02_trimmed/U1_L001_R1.fastq.gz \\
    -O 02_trimmed/U1_L001_R2.fastq.gz \\
    --detect_adapter_for_pe \\
    --qualified_quality_phred 20 \\
    --thread {N_THREADS}
""")

### 1.3 Lane Merging

In [ ]:
# === Lane Merging ===
#
# Merge L001 and L002 files for each sample.
# Result: 120 files (60 samples × 2 reads)
#
# Shell command:
# cat U1_L001_R1.fastq.gz U1_L002_R1.fastq.gz > U1_R1.fastq.gz

RUN_MERGE = False  # Set to True to run

def merge_lanes(sample_id):
    """
    Merge lane files for a single sample.
    """
    for read in ['R1', 'R2']:
        l001 = DIRS['trimmed'] / f"{sample_id}_L001_{read}.fastq.gz"
        l002 = DIRS['trimmed'] / f"{sample_id}_L002_{read}.fastq.gz"
        merged = DIRS['merged'] / f"{sample_id}_{read}.fastq.gz"
        
        # Use cat (gzipped files can be concatenated directly)
        cmd = f'cat "{l001}" "{l002}" > "{merged}"'
        subprocess.run(cmd, shell=True, check=True)
    
    return sample_id


if RUN_MERGE:
    samples = metadata['sample_id'].tolist()
    print(f"Merging lanes for {len(samples)} samples...")
    
    results = Parallel(n_jobs=N_THREADS)(
        delayed(merge_lanes)(s) for s in tqdm(samples, desc="Merging")
    )
    
    print(f"\nMerged files saved to: {DIRS['merged']}")
else:
    print("Skipping lane merging (set RUN_MERGE = True to run)")
    print("\nExample commands:")
    print("cat 02_trimmed/U1_L001_R1.fastq.gz 02_trimmed/U1_L002_R1.fastq.gz > 03_merged/U1_R1.fastq.gz")
    print("cat 02_trimmed/U1_L001_R2.fastq.gz 02_trimmed/U1_L002_R2.fastq.gz > 03_merged/U1_R2.fastq.gz")

### 1.4 rRNA Removal with SortMeRNA

In [ ]:
# === rRNA Removal with SortMeRNA ===
#
# SortMeRNA filters out ribosomal RNA reads.
# This is critical for bacterial RNA-seq even after rRNA depletion during library prep.
#
# IMPORTANT: Memory-intensive! Use max 3-4 threads (~15GB RAM each)
# IMPORTANT: This step takes ~45-60 min PER SAMPLE. Running 60 samples = ~2.5 days!
#            DO NOT run in the notebook kernel - use the generated shell script instead.
#
# Shell command:
# sortmerna \
#     --ref silva-bac-16s-id90.fasta \
#     --ref silva-bac-23s-id98.fasta \
#     --reads U1_R1.fastq.gz --reads U1_R2.fastq.gz \
#     --paired_in --out2 \
#     --aligned rRNA/U1 \
#     --other non_rRNA/U1 \
#     --fastx --threads 4

GENERATE_SORTMERNA_SCRIPT = True  # Generate shell script for external execution

# SortMeRNA database paths (adjust based on your conda environment)
SORTMERNA_DB = os.environ.get('CONDA_PREFIX', '') + '/share/sortmerna/rRNA_databases'

if GENERATE_SORTMERNA_SCRIPT:
    samples = metadata['sample_id'].tolist()
    script_path = DIRS['logs'] / "run_sortmerna_all.sh"
    
    with open(script_path, "w") as f:
        f.write("#!/bin/bash\n")
        f.write("# ==============================================\n")
        f.write("# SortMeRNA Batch Script - PCC 11901 RNA-seq\n")
        f.write("# ==============================================\n")
        f.write("# IMPORTANT: Run this in a separate terminal, NOT in Jupyter!\n")
        f.write("# Estimated runtime: ~45-60 hours (60 samples × ~1 hour each)\n")
        f.write("#\n")
        f.write("# Usage:\n")
        f.write("#   conda activate pcc11901_rnaseq\n")
        f.write(f"#   bash {script_path}\n")
        f.write("#\n")
        f.write("# To resume after interruption, comment out completed samples.\n")
        f.write("# ==============================================\n\n")
        f.write("set -e  # Exit on error\n\n")
        f.write(f"# SortMeRNA database path\n")
        f.write(f"SORTMERNA_DB=\"$CONDA_PREFIX/share/sortmerna/rRNA_databases\"\n\n")
        f.write(f"# Verify database exists\n")
        f.write(f"if [ ! -d \"$SORTMERNA_DB\" ]; then\n")
        f.write(f"    echo \"ERROR: SortMeRNA database not found at $SORTMERNA_DB\"\n")
        f.write(f"    exit 1\n")
        f.write(f"fi\n\n")
        f.write(f"echo \"Starting SortMeRNA processing for {len(samples)} samples...\"\n")
        f.write(f"echo \"Start time: $(date)\"\n\n")
        
        for i, sample in enumerate(samples, 1):
            r1 = DIRS['merged'] / f"{sample}_R1.fastq.gz"
            r2 = DIRS['merged'] / f"{sample}_R2.fastq.gz"
            aligned_prefix = DIRS['rrna_filtered'] / 'rRNA' / sample
            other_prefix = DIRS['rrna_filtered'] / 'non_rRNA' / sample
            workdir = DIRS['rrna_filtered'] / f'workdir_{sample}'
            log_file = DIRS['logs'] / f'{sample}_sortmerna.log'
            
            f.write(f"# === Sample {i}/{len(samples)}: {sample} ===\n")
            f.write(f"echo \"[{i}/{len(samples)}] Processing {sample}...\"\n")
            f.write(f"mkdir -p \"{workdir}\"\n")
            f.write(f"sortmerna \\\n")
            f.write(f"    --ref \"$SORTMERNA_DB/silva-bac-16s-id90.fasta\" \\\n")
            f.write(f"    --ref \"$SORTMERNA_DB/silva-bac-23s-id98.fasta\" \\\n")
            f.write(f"    --reads \"{r1}\" --reads \"{r2}\" \\\n")
            f.write(f"    --paired_in --out2 \\\n")
            f.write(f"    --aligned \"{aligned_prefix}\" \\\n")
            f.write(f"    --other \"{other_prefix}\" \\\n")
            f.write(f"    --fastx \\\n")
            f.write(f"    --threads {SORTMERNA_THREADS} \\\n")
            f.write(f"    --workdir \"{workdir}\" \\\n")
            f.write(f"    2>&1 | tee \"{log_file}\"\n")
            f.write(f"rm -rf \"{workdir}\"\n")
            f.write(f"echo \"  Completed {sample} at $(date)\"\n\n")
        
        f.write("echo \"==============================================\"\n")
        f.write("echo \"SortMeRNA processing complete!\"\n")
        f.write("echo \"End time: $(date)\"\n")
        f.write("echo \"==============================================\"\n")
    
    # Make executable
    os.chmod(script_path, 0o755)
    
    print(f"Generated SortMeRNA batch script: {script_path}")
    print(f"\nTo run (in a separate terminal):")
    print(f"  conda activate pcc11901_rnaseq")
    print(f"  bash {script_path}")
    print(f"\nEstimated runtime: ~45-60 hours for {len(samples)} samples")
    print("TIP: Use 'screen' or 'tmux' to keep it running if you disconnect.")
else:
    print("Skipping SortMeRNA script generation (set GENERATE_SORTMERNA_SCRIPT = True)")
    print("\nExample command for single sample:")
    print(f"""
sortmerna \\
    --ref $CONDA_PREFIX/share/sortmerna/rRNA_databases/silva-bac-16s-id90.fasta \\
    --ref $CONDA_PREFIX/share/sortmerna/rRNA_databases/silva-bac-23s-id98.fasta \\
    --reads 03_merged/U1_R1.fastq.gz --reads 03_merged/U1_R2.fastq.gz \\
    --paired_in --out2 \\
    --aligned 04_rRNA_filtered/rRNA/U1 \\
    --other 04_rRNA_filtered/non_rRNA/U1 \\
    --fastx --threads {SORTMERNA_THREADS}
""")

### 1.5 Post-Processing QC

In [ ]:
# === FastQC on Trimmed/Filtered Reads ===
#
# Run FastQC on the rRNA-filtered reads to verify quality improvement.

RUN_FASTQC_TRIMMED = False  # Set to True to run

if RUN_FASTQC_TRIMMED:
    output_dir = DIRS['qc'] / 'fastqc_trimmed'
    input_dir = DIRS['rrna_filtered'] / 'non_rRNA'
    
    cmd = f"fastqc -t {N_THREADS} -o {output_dir} {input_dir}/*.fastq.gz"
    print(f"Running FastQC on filtered reads...")
    print(f"Command: {cmd}\n")
    
    !{cmd}
    
    print("\nFastQC complete!")
else:
    print("Skipping post-filter FastQC (set RUN_FASTQC_TRIMMED = True to run)")

In [ ]:
# === MultiQC Summary Report ===
#
# Aggregate all QC reports into a single interactive report.

RUN_MULTIQC = False  # Set to True to run

if RUN_MULTIQC:
    output_dir = DIRS['qc'] / 'multiqc_reports'
    
    cmd = f"""
multiqc \\
    {DIRS['qc']}/fastqc_raw \\
    {DIRS['qc']}/fastqc_trimmed \\
    {DIRS['trimmed']} \\
    -o {output_dir} \\
    --filename multiqc_report \\
    --force
"""
    print("Running MultiQC...")
    !{cmd}
    
    print(f"\nReport saved to: {output_dir}/multiqc_report.html")
else:
    print("Skipping MultiQC (set RUN_MULTIQC = True to run)")
    print(f"\nTo run manually:\nmultiqc {DIRS['qc']}/ -o {DIRS['qc']}/multiqc_reports/")

---
## Phase 2: Quantification and Alignment

### 2.1 Extract Transcriptome (gffread)

In [ ]:
# === Extract Transcriptome with gffread ===
#
# Create a FASTA file of transcript sequences from the genome + GTF.
# This is required for Salmon indexing.
#
# Shell command:
# gffread -w transcriptome.fa -g genome.fna annotation.gtf

RUN_GFFREAD = False  # Set to True to run

if RUN_GFFREAD:
    cmd = f"""
gffread \\
    -w "{TRANSCRIPTOME_FASTA}" \\
    -g "{GENOME_FASTA}" \\
    "{GTF_FILE}"
"""
    print("Extracting transcriptome...")
    print(f"Command: {cmd}")
    
    !{cmd}
    
    # Check result
    if TRANSCRIPTOME_FASTA.exists():
        n_transcripts = !grep -c "^>" "{TRANSCRIPTOME_FASTA}"
        print(f"\nTranscriptome created: {TRANSCRIPTOME_FASTA}")
        print(f"Number of transcripts: {n_transcripts[0]}")
    else:
        print("[ERROR] Transcriptome extraction failed!")
else:
    print("Skipping transcriptome extraction (set RUN_GFFREAD = True to run)")
    print(f"\nTo run manually:")
    print(f'gffread -w "{TRANSCRIPTOME_FASTA}" -g "{GENOME_FASTA}" "{GTF_FILE}"')

### 2.2 Salmon Index and Quantification

In [ ]:
# === Build Salmon Index ===
#
# Using decoy-aware mapping for better accuracy:
# - Target: transcriptome sequences
# - Decoy: whole genome (prevents false quantification of intergenic reads)
#
# Shell commands:
# grep "^>" genome.fna | cut -d " " -f 1 | sed 's/>//g' > decoys.txt
# cat transcriptome.fa genome.fna > gentrome.fa
# salmon index -t gentrome.fa -d decoys.txt -i salmon_index -p 8

RUN_SALMON_INDEX = False  # Set to True to run

SALMON_INDEX = DIRS['salmon'] / 'index'
GENTROME = DIRS['genome'] / 'gentrome.fa'
DECOYS = DIRS['genome'] / 'decoys.txt'

if RUN_SALMON_INDEX:
    print("Building Salmon index with decoy-aware mapping...\n")
    
    # Step 1: Create decoys.txt (chromosome names)
    print("Step 1: Creating decoys list...")
    cmd_decoys = f'grep "^>" "{GENOME_FASTA}" | cut -d " " -f 1 | sed "s/>//g" > "{DECOYS}"'
    !{cmd_decoys}
    print(f"  Decoys file: {DECOYS}")
    
    # Step 2: Create gentrome (transcriptome + genome)
    print("\nStep 2: Creating gentrome...")
    cmd_gentrome = f'cat "{TRANSCRIPTOME_FASTA}" "{GENOME_FASTA}" > "{GENTROME}"'
    !{cmd_gentrome}
    print(f"  Gentrome file: {GENTROME}")
    
    # Step 3: Build Salmon index
    print(f"\nStep 3: Building Salmon index (this may take a few minutes)...")
    cmd_index = f"""
salmon index \\
    -t "{GENTROME}" \\
    -d "{DECOYS}" \\
    -i "{SALMON_INDEX}" \\
    -p {SALMON_THREADS}
"""
    print(f"Command: {cmd_index}")
    !{cmd_index}
    
    print(f"\nSalmon index created: {SALMON_INDEX}")
else:
    print("Skipping Salmon index build (set RUN_SALMON_INDEX = True to run)")
    print("\nCommands to run manually:")
    print(f'grep "^>" "{GENOME_FASTA}" | cut -d " " -f 1 | sed "s/>//g" > "{DECOYS}"')
    print(f'cat "{TRANSCRIPTOME_FASTA}" "{GENOME_FASTA}" > "{GENTROME}"')
    print(f'salmon index -t "{GENTROME}" -d "{DECOYS}" -i "{SALMON_INDEX}" -p {SALMON_THREADS}')

In [ ]:
# === Salmon Quantification ===
#
# Quantify transcript abundance for each sample.
# Salmon auto-detects library type (strandedness).
#
# Shell command:
# salmon quant -i salmon_index -l A \
#     -1 U1_R1.fastq.gz -2 U1_R2.fastq.gz \
#     -o quants/U1 --validateMappings --gcBias -p 10

RUN_SALMON_QUANT = False  # Set to True to run

def run_salmon_quant(sample_id):
    """
    Run Salmon quantification on a single sample.
    Uses rRNA-filtered reads from 04_rRNA_filtered/non_rRNA/
    """
    # Input files (from SortMeRNA output)
    r1 = DIRS['rrna_filtered'] / 'non_rRNA' / f"{sample_id}_fwd.fq.gz"
    r2 = DIRS['rrna_filtered'] / 'non_rRNA' / f"{sample_id}_rev.fq.gz"
    
    # Output directory
    output_dir = DIRS['salmon'] / 'quants' / sample_id
    
    cmd = f"""
salmon quant \\
    -i "{SALMON_INDEX}" \\
    -l A \\
    -1 "{r1}" \\
    -2 "{r2}" \\
    -o "{output_dir}" \\
    --validateMappings \\
    --gcBias \\
    --seqBias \\
    -p {SALMON_THREADS}
"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return sample_id


if RUN_SALMON_QUANT:
    samples = metadata['sample_id'].tolist()
    print(f"Running Salmon quantification on {len(samples)} samples...")
    print(f"Output: {DIRS['salmon']}/quants/")
    
    # Run sequentially (Salmon already uses multiple threads)
    for sample in tqdm(samples, desc="Salmon quant"):
        run_salmon_quant(sample)
    
    print("\nSalmon quantification complete!")
else:
    print("Skipping Salmon quantification (set RUN_SALMON_QUANT = True to run)")
    print("\nExample command for single sample:")
    print(f"""
salmon quant \\
    -i "{SALMON_INDEX}" \\
    -l A \\
    -1 04_rRNA_filtered/non_rRNA/U1_fwd.fq.gz \\
    -2 04_rRNA_filtered/non_rRNA/U1_rev.fq.gz \\
    -o 05_salmon/quants/U1 \\
    --validateMappings --gcBias -p {SALMON_THREADS}
""")

### 2.3 Bowtie2 Alignment (for Visualization)

In [ ]:
# === Build Bowtie2 Index ===
#
# Build genome index for read alignment.
# This is needed to create BAM files for IGV visualization.
#
# Shell command:
# bowtie2-build genome.fna bowtie2_index/pcc11901

RUN_BOWTIE2_INDEX = False  # Set to True to run

BOWTIE2_INDEX = DIRS['alignment'] / 'bowtie2_index' / 'pcc11901'

if RUN_BOWTIE2_INDEX:
    print("Building Bowtie2 index...")
    
    cmd = f'bowtie2-build --threads {N_THREADS} "{GENOME_FASTA}" "{BOWTIE2_INDEX}"'
    print(f"Command: {cmd}\n")
    
    !{cmd}
    
    print(f"\nBowtie2 index created: {BOWTIE2_INDEX}")
else:
    print("Skipping Bowtie2 index (set RUN_BOWTIE2_INDEX = True to run)")
    print(f'\nTo run manually:\nbowtie2-build --threads {N_THREADS} "{GENOME_FASTA}" "{BOWTIE2_INDEX}"')

In [ ]:
# === Bowtie2 Alignment ===
#
# Align reads to genome and create sorted BAM files.
#
# Shell command:
# bowtie2 -x bowtie2_index/pcc11901 -1 R1.fq.gz -2 R2.fq.gz -p 8 | \
#     samtools view -bS - | samtools sort -o sample.bam
# samtools index sample.bam

RUN_BOWTIE2_ALIGN = False  # Set to True to run

def run_bowtie2_align(sample_id):
    """
    Align reads with Bowtie2 and create sorted BAM.
    """
    r1 = DIRS['rrna_filtered'] / 'non_rRNA' / f"{sample_id}_fwd.fq.gz"
    r2 = DIRS['rrna_filtered'] / 'non_rRNA' / f"{sample_id}_rev.fq.gz"
    
    bam_file = DIRS['alignment'] / 'bam' / f"{sample_id}.bam"
    
    cmd = f"""
bowtie2 \\
    -x "{BOWTIE2_INDEX}" \\
    -1 "{r1}" \\
    -2 "{r2}" \\
    -p {N_THREADS} \\
    2> "{DIRS['logs']}/{sample_id}_bowtie2.log" | \
samtools view -bS - | \
samtools sort -@ 4 -o "{bam_file}" -

samtools index "{bam_file}"
"""
    subprocess.run(cmd, shell=True, check=True)
    return sample_id


if RUN_BOWTIE2_ALIGN:
    samples = metadata['sample_id'].tolist()
    print(f"Running Bowtie2 alignment on {len(samples)} samples...")
    
    for sample in tqdm(samples, desc="Aligning"):
        run_bowtie2_align(sample)
    
    print(f"\nBAM files saved to: {DIRS['alignment']}/bam/")
else:
    print("Skipping Bowtie2 alignment (set RUN_BOWTIE2_ALIGN = True to run)")
    print("\nExample command:")
    print(f"""
bowtie2 -x "{BOWTIE2_INDEX}" \\
    -1 04_rRNA_filtered/non_rRNA/U1_fwd.fq.gz \\
    -2 04_rRNA_filtered/non_rRNA/U1_rev.fq.gz \\
    -p {N_THREADS} | samtools view -bS - | samtools sort -o 06_alignment/bam/U1.bam -
samtools index 06_alignment/bam/U1.bam
""")

In [ ]:
# === Create bigWig Coverage Tracks ===
#
# Convert BAM to normalized bigWig for IGV visualization.
# Using CPM (counts per million) normalization.
#
# Shell command:
# bamCoverage -b sample.bam -o sample.bw --normalizeUsing CPM -p 8

RUN_BIGWIG = False  # Set to True to run

def create_bigwig(sample_id):
    """
    Create CPM-normalized bigWig from BAM.
    """
    bam_file = DIRS['alignment'] / 'bam' / f"{sample_id}.bam"
    bigwig_file = DIRS['alignment'] / 'bigwig' / f"{sample_id}.bw"
    
    cmd = f"""
bamCoverage \\
    -b "{bam_file}" \\
    -o "{bigwig_file}" \\
    --normalizeUsing CPM \\
    -p {N_THREADS}
"""
    subprocess.run(cmd, shell=True, check=True, capture_output=True)
    return sample_id


if RUN_BIGWIG:
    samples = metadata['sample_id'].tolist()
    print(f"Creating bigWig files for {len(samples)} samples...")
    
    for sample in tqdm(samples, desc="bigWig"):
        create_bigwig(sample)
    
    print(f"\nbigWig files saved to: {DIRS['alignment']}/bigwig/")
else:
    print("Skipping bigWig creation (set RUN_BIGWIG = True to run)")
    print(f"\nTo run manually:\nbamCoverage -b 06_alignment/bam/U1.bam -o 06_alignment/bigwig/U1.bw --normalizeUsing CPM -p {N_THREADS}")

---
## Phase 3: Differential Expression Analysis

### 3.1 Import Salmon Counts with tximport

In [ ]:
# === Setup R Environment via rpy2 ===

# Import R packages
base = importr('base')
stats = importr('stats')

# Load Bioconductor packages
try:
    tximport = importr('tximport')
    deseq2 = importr('DESeq2')
    print("[OK] tximport and DESeq2 loaded")
except Exception as e:
    print(f"[ERROR] Could not load R packages: {e}")
    print("Install with: R -e 'BiocManager::install(c(\"tximport\", \"DESeq2\"))'")

In [ ]:
# === Load Salmon Counts ===
#
# Use tximport to aggregate transcript-level counts to gene level.
# This is the recommended way to import Salmon output into DESeq2.

LOAD_COUNTS = False  # Set to True after Salmon quant is complete

if LOAD_COUNTS:
    # Create tx2gene mapping from GTF
    print("Creating transcript-to-gene mapping...")
    
    # Parse GTF to get transcript -> gene mapping
    # Note: NCBI RefSeq GTF files may use different attribute formats
    tx2gene = []
    with open(GTF_FILE, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9:
                continue
            
            # Only process CDS or exon features (which have transcript associations)
            feature_type = fields[2]
            if feature_type not in ['CDS', 'exon', 'transcript']:
                continue
                
            attrs = fields[8]
            
            # Try multiple patterns for transcript_id (RefSeq GTFs vary)
            tx_match = re.search(r'transcript_id "([^"]+)"', attrs)
            if not tx_match:
                tx_match = re.search(r'transcript_id=([^;]+)', attrs)
            
            # Try multiple patterns for gene_id
            gene_match = re.search(r'gene_id "([^"]+)"', attrs)
            if not gene_match:
                gene_match = re.search(r'gene_id=([^;]+)', attrs)
            if not gene_match:
                # Fallback: try locus_tag
                gene_match = re.search(r'locus_tag "([^"]+)"', attrs)
            
            if tx_match and gene_match:
                tx2gene.append({
                    'transcript_id': tx_match.group(1),
                    'gene_id': gene_match.group(1)
                })
    
    tx2gene_df = pd.DataFrame(tx2gene).drop_duplicates()
    
    # === VALIDATION: Check tx2gene parsing succeeded ===
    if len(tx2gene_df) == 0:
        print("\n[ERROR] tx2gene parsing failed! No transcript-gene mappings found.")
        print("Debugging info - first 5 lines of GTF attributes:")
        with open(GTF_FILE, 'r') as f:
            for i, line in enumerate(f):
                if not line.startswith('#') and i < 10:
                    fields = line.strip().split('\t')
                    if len(fields) >= 9:
                        print(f"  {fields[8][:200]}...")
        raise ValueError(
            "tx2gene parsing failed! Check if your GTF uses 'gene_id'/'transcript_id' "
            "or alternative formats like 'gene', 'locus_tag', etc. "
            "Preview the GTF attributes above to debug."
        )
    
    tx2gene_file = DIRS['deseq2'] / 'tx2gene.csv'
    tx2gene_df.to_csv(tx2gene_file, index=False)
    print(f"  tx2gene mapping: {len(tx2gene_df)} transcripts -> {tx2gene_df['gene_id'].nunique()} genes")
    
    # Get Salmon quant files
    samples = metadata['sample_id'].tolist()
    quant_files = [str(DIRS['salmon'] / 'quants' / s / 'quant.sf') for s in samples]
    
    # Check that files exist
    missing = [f for f in quant_files if not Path(f).exists()]
    if missing:
        print(f"[WARNING] Missing quant files: {len(missing)}")
        print(f"  First missing: {missing[0]}")
    else:
        print(f"  Found all {len(quant_files)} quant.sf files")
    
    # Import with tximport via R
    print("\nRunning tximport...")
    
    ro.r(f'''
    library(tximport)
    
    # Read tx2gene
    tx2gene <- read.csv("{tx2gene_file}")
    
    # Sample files
    files <- c({', '.join([f'"{f}"' for f in quant_files])})
    names(files) <- c({', '.join([f'"{s}"' for s in samples])})
    
    # Import
    txi <- tximport(files, type="salmon", tx2gene=tx2gene)
    
    # Save counts
    counts <- as.data.frame(txi$counts)
    write.csv(counts, "{DIRS['deseq2']}/counts_matrix.csv")
    ''')
    
    # Load counts into Python
    counts_df = pd.read_csv(DIRS['deseq2'] / 'counts_matrix.csv', index_col=0)
    print(f"\nCounts matrix: {counts_df.shape[0]} genes × {counts_df.shape[1]} samples")
    counts_df.head()
else:
    print("Skipping count loading (set LOAD_COUNTS = True after Salmon quant is complete)")

### 3.2 DESeq2 Analysis

In [ ]:
# === DESeq2 Analysis - Group 1 (Nutrients) ===
#
# Compare each nutrient condition vs Control (U4, U5, U6)
#
# Conditions:
# - Glycerol_0.75pct
# - Low_Nitrogen, High_Nitrogen
# - Low_Phosphate, High_Phosphate
# - Ammonia, Urea

RUN_DESEQ2_GROUP1 = False  # Set to True to run

if RUN_DESEQ2_GROUP1:
    print("Running DESeq2 for Group 1 (Nutrients)...\n")
    
    # Filter metadata for Group 1
    group1_meta = metadata[metadata['group'] == 1].copy()
    group1_samples = group1_meta['sample_id'].tolist()
    
    # Save metadata for R
    group1_meta_file = DIRS['deseq2'] / 'group1_metadata.csv'
    group1_meta.to_csv(group1_meta_file, index=False)
    
    # Run DESeq2 in R
    ro.r(f'''
    library(DESeq2)
    library(tximport)
    
    # Load metadata
    coldata <- read.csv("{group1_meta_file}")
    rownames(coldata) <- coldata$sample_id
    coldata$condition <- factor(coldata$condition)
    coldata$condition <- relevel(coldata$condition, ref="Control_MAD")
    
    # Load counts (subset to Group 1 samples)
    counts <- read.csv("{DIRS['deseq2']}/counts_matrix.csv", row.names=1)
    counts <- counts[, coldata$sample_id]
    counts <- round(counts)  # DESeq2 requires integers
    
    # Create DESeq2 object
    dds <- DESeqDataSetFromMatrix(
        countData = counts,
        colData = coldata,
        design = ~ condition
    )
    
    # Filter low counts
    keep <- rowSums(counts(dds) >= 10) >= 3
    dds <- dds[keep,]
    
    # Run DESeq2
    dds <- DESeq(dds)
    
    # Save normalized counts
    norm_counts <- as.data.frame(counts(dds, normalized=TRUE))
    write.csv(norm_counts, "{DIRS['deseq2']}/group1_normalized_counts.csv")
    
    # Extract results for each comparison
    conditions <- levels(coldata$condition)
    conditions <- conditions[conditions != "Control_MAD"]
    
    for (cond in conditions) {{
        res <- results(dds, contrast=c("condition", cond, "Control_MAD"))
        res <- as.data.frame(res)
        res$gene_id <- rownames(res)
        write.csv(res, paste0("{DIRS['deseq2']}/group1_", cond, "_vs_Control.csv"), row.names=FALSE)
        
        # Summary
        sig <- sum(res$padj < 0.05 & abs(res$log2FoldChange) > 1, na.rm=TRUE)
        cat(paste0(cond, " vs Control: ", sig, " DE genes\n"))
    }}
    ''')
    
    print("\nGroup 1 analysis complete!")
    print(f"Results saved to: {DIRS['deseq2']}/")
else:
    print("Skipping DESeq2 Group 1 (set RUN_DESEQ2_GROUP1 = True to run)")

In [ ]:
# === DESeq2 Analysis - Group 2 (Environmental) ===
#
# Compare each environmental condition vs Control (U1, U2, U3)
#
# Conditions:
# - High_NaCl_9pct, H2O2_0.005pct
# - Atmospheric_CO2, High_CO2_8pct
# - High_Temp_38C
# - Low_Light_15uE, High_Light

RUN_DESEQ2_GROUP2 = False  # Set to True to run

if RUN_DESEQ2_GROUP2:
    print("Running DESeq2 for Group 2 (Environmental)...\n")
    
    # Filter metadata for Group 2
    group2_meta = metadata[metadata['group'] == 2].copy()
    
    # Save metadata for R
    group2_meta_file = DIRS['deseq2'] / 'group2_metadata.csv'
    group2_meta.to_csv(group2_meta_file, index=False)
    
    # Run DESeq2 in R
    ro.r(f'''
    library(DESeq2)
    
    # Load metadata
    coldata <- read.csv("{group2_meta_file}")
    rownames(coldata) <- coldata$sample_id
    coldata$condition <- factor(coldata$condition)
    coldata$condition <- relevel(coldata$condition, ref="Control_MAD")
    
    # Load counts (subset to Group 2 samples)
    counts <- read.csv("{DIRS['deseq2']}/counts_matrix.csv", row.names=1)
    counts <- counts[, coldata$sample_id]
    counts <- round(counts)
    
    # Create DESeq2 object
    dds <- DESeqDataSetFromMatrix(
        countData = counts,
        colData = coldata,
        design = ~ condition
    )
    
    # Filter low counts
    keep <- rowSums(counts(dds) >= 10) >= 3
    dds <- dds[keep,]
    
    # Run DESeq2
    dds <- DESeq(dds)
    
    # Save normalized counts
    norm_counts <- as.data.frame(counts(dds, normalized=TRUE))
    write.csv(norm_counts, "{DIRS['deseq2']}/group2_normalized_counts.csv")
    
    # Extract results for each comparison
    conditions <- levels(coldata$condition)
    conditions <- conditions[conditions != "Control_MAD"]
    
    for (cond in conditions) {{
        res <- results(dds, contrast=c("condition", cond, "Control_MAD"))
        res <- as.data.frame(res)
        res$gene_id <- rownames(res)
        write.csv(res, paste0("{DIRS['deseq2']}/group2_", cond, "_vs_Control.csv"), row.names=FALSE)
        
        sig <- sum(res$padj < 0.05 & abs(res$log2FoldChange) > 1, na.rm=TRUE)
        cat(paste0(cond, " vs Control: ", sig, " DE genes\n"))
    }}
    ''')
    
    print("\nGroup 2 analysis complete!")
else:
    print("Skipping DESeq2 Group 2 (set RUN_DESEQ2_GROUP2 = True to run)")

In [ ]:
# === DESeq2 Analysis - Group 3 (Circadian) ===
#
# Time-series analysis using DESeq2 LRT (Likelihood Ratio Test)
# Tests for genes with significant changes across timepoints.
#
# Timepoints:
# - T1 (Light), T2 (Dark), T3 (Light), T4 (Dark)

RUN_DESEQ2_GROUP3 = False  # Set to True to run

if RUN_DESEQ2_GROUP3:
    print("Running DESeq2 for Group 3 (Circadian - LRT)...\n")
    
    # Filter metadata for Group 3
    group3_meta = metadata[metadata['group'] == 3].copy()
    
    # Add timepoint numeric for trend analysis
    timepoint_map = {'T1_Light': 1, 'T2_Dark': 2, 'T3_Light': 3, 'T4_Dark': 4}
    group3_meta['timepoint'] = group3_meta['condition'].map(timepoint_map)
    
    # Save metadata for R
    group3_meta_file = DIRS['deseq2'] / 'group3_metadata.csv'
    group3_meta.to_csv(group3_meta_file, index=False)
    
    # Run DESeq2 LRT in R
    ro.r(f'''
    library(DESeq2)
    
    # Load metadata
    coldata <- read.csv("{group3_meta_file}")
    rownames(coldata) <- coldata$sample_id
    coldata$condition <- factor(coldata$condition, 
                                 levels=c("T1_Light", "T2_Dark", "T3_Light", "T4_Dark"))
    
    # Load counts
    counts <- read.csv("{DIRS['deseq2']}/counts_matrix.csv", row.names=1)
    counts <- counts[, coldata$sample_id]
    counts <- round(counts)
    
    # Create DESeq2 object
    dds <- DESeqDataSetFromMatrix(
        countData = counts,
        colData = coldata,
        design = ~ condition
    )
    
    # Filter low counts
    keep <- rowSums(counts(dds) >= 10) >= 3
    dds <- dds[keep,]
    
    # Run DESeq2 with LRT (tests for ANY difference across timepoints)
    dds <- DESeq(dds, test="LRT", reduced=~1)
    
    # Save normalized counts
    norm_counts <- as.data.frame(counts(dds, normalized=TRUE))
    write.csv(norm_counts, "{DIRS['deseq2']}/group3_normalized_counts.csv")
    
    # LRT results (genes changing over time)
    res_lrt <- results(dds)
    res_lrt <- as.data.frame(res_lrt)
    res_lrt$gene_id <- rownames(res_lrt)
    write.csv(res_lrt, "{DIRS['deseq2']}/group3_circadian_LRT.csv", row.names=FALSE)
    
    sig <- sum(res_lrt$padj < 0.05, na.rm=TRUE)
    cat(paste0("Genes with significant circadian variation: ", sig, "\n"))
    
    # Also do pairwise comparisons
    # T2 (Dark) vs T1 (Light)
    dds_wald <- DESeq(dds, test="Wald")
    
    res_t2t1 <- results(dds_wald, contrast=c("condition", "T2_Dark", "T1_Light"))
    write.csv(as.data.frame(res_t2t1), "{DIRS['deseq2']}/group3_T2_Dark_vs_T1_Light.csv")
    
    res_t3t2 <- results(dds_wald, contrast=c("condition", "T3_Light", "T2_Dark"))
    write.csv(as.data.frame(res_t3t2), "{DIRS['deseq2']}/group3_T3_Light_vs_T2_Dark.csv")
    
    res_t4t3 <- results(dds_wald, contrast=c("condition", "T4_Dark", "T3_Light"))
    write.csv(as.data.frame(res_t4t3), "{DIRS['deseq2']}/group3_T4_Dark_vs_T3_Light.csv")
    ''')
    
    print("\nGroup 3 analysis complete!")
else:
    print("Skipping DESeq2 Group 3 (set RUN_DESEQ2_GROUP3 = True to run)")

---
## Phase 4: Visualization

### 4.1 QC Plots (PCA, Sample Correlation)

In [ ]:
# === PCA Plot ===
#
# Visualize sample clustering using principal component analysis.

PLOT_PCA = False  # Set to True after counts are loaded

if PLOT_PCA:
    # Load normalized counts
    norm_counts = pd.read_csv(DIRS['deseq2'] / 'counts_matrix.csv', index_col=0)
    
    # Log transform
    log_counts = np.log2(norm_counts + 1)
    
    # PCA
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(log_counts.T)  # Samples as rows
    
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(scaled_data)
    
    # Create DataFrame for plotting
    pca_df = pd.DataFrame({
        'PC1': pca_result[:, 0],
        'PC2': pca_result[:, 1],
        'sample_id': norm_counts.columns
    })
    pca_df = pca_df.merge(metadata, on='sample_id')
    
    # Plot with Plotly
    fig = px.scatter(
        pca_df, x='PC1', y='PC2',
        color='condition',
        symbol='group',
        hover_data=['sample_id'],
        title='PCA: All Samples',
        labels={
            'PC1': f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
            'PC2': f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)'
        }
    )
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(height=600, width=900)
    fig.show()
    
    # Save
    fig.write_html(DIRS['figures'] / 'qc_plots' / 'pca_all_samples.html')
    print(f"\nSaved: {DIRS['figures']}/qc_plots/pca_all_samples.html")
else:
    print("Skipping PCA plot (set PLOT_PCA = True after loading counts)")

In [ ]:
# === Sample Correlation Heatmap ===

PLOT_CORRELATION = False  # Set to True after counts are loaded

if PLOT_CORRELATION:
    # Load normalized counts
    norm_counts = pd.read_csv(DIRS['deseq2'] / 'counts_matrix.csv', index_col=0)
    
    # Calculate correlation
    log_counts = np.log2(norm_counts + 1)
    corr_matrix = log_counts.corr(method='pearson')
    
    # Create annotation colors
    sample_order = metadata['sample_id'].tolist()
    corr_matrix = corr_matrix.loc[sample_order, sample_order]
    
    # Plot
    fig = px.imshow(
        corr_matrix,
        labels=dict(color="Pearson r"),
        x=corr_matrix.columns,
        y=corr_matrix.index,
        color_continuous_scale='RdBu_r',
        zmin=0.8, zmax=1.0,
        title='Sample Correlation Matrix'
    )
    fig.update_layout(height=800, width=900)
    fig.show()
    
    # Save
    fig.write_html(DIRS['figures'] / 'qc_plots' / 'correlation_heatmap.html')
else:
    print("Skipping correlation heatmap (set PLOT_CORRELATION = True after loading counts)")

### 4.2 Volcano Plots

In [ ]:
# === Volcano Plot Function ===

def create_volcano_plot(results_file, title, output_path=None, 
                        log2fc_threshold=1, padj_threshold=0.05):
    """
    Create an interactive volcano plot from DESeq2 results.
    
    Parameters:
    -----------
    results_file : str or Path
        Path to DESeq2 results CSV
    title : str
        Plot title
    output_path : Path, optional
        Path to save HTML output
    log2fc_threshold : float
        Log2 fold change threshold for significance
    padj_threshold : float
        Adjusted p-value threshold
    """
    # Load results
    df = pd.read_csv(results_file)
    
    # Remove NA values
    df = df.dropna(subset=['log2FoldChange', 'padj'])
    
    # Add -log10(padj)
    df['neg_log10_padj'] = -np.log10(df['padj'])
    
    # Classify genes
    conditions = [
        (df['padj'] < padj_threshold) & (df['log2FoldChange'] > log2fc_threshold),
        (df['padj'] < padj_threshold) & (df['log2FoldChange'] < -log2fc_threshold),
    ]
    choices = ['Up', 'Down']
    df['regulation'] = np.select(conditions, choices, default='NS')
    
    # Count DE genes
    n_up = (df['regulation'] == 'Up').sum()
    n_down = (df['regulation'] == 'Down').sum()
    
    # Color mapping
    color_map = {'Up': 'red', 'Down': 'blue', 'NS': 'gray'}
    
    # Create plot
    fig = px.scatter(
        df, x='log2FoldChange', y='neg_log10_padj',
        color='regulation',
        color_discrete_map=color_map,
        hover_data=['gene_id', 'baseMean', 'padj'],
        title=f"{title}<br><sub>Up: {n_up}, Down: {n_down}</sub>",
        labels={
            'log2FoldChange': 'Log2 Fold Change',
            'neg_log10_padj': '-Log10(adjusted p-value)'
        }
    )
    
    # Add threshold lines
    fig.add_hline(y=-np.log10(padj_threshold), line_dash="dash", line_color="gray")
    fig.add_vline(x=log2fc_threshold, line_dash="dash", line_color="gray")
    fig.add_vline(x=-log2fc_threshold, line_dash="dash", line_color="gray")
    
    fig.update_traces(marker=dict(size=5, opacity=0.6))
    fig.update_layout(height=600, width=800)
    
    if output_path:
        fig.write_html(output_path)
        print(f"Saved: {output_path}")
    
    return fig


print("Volcano plot function defined.")

In [ ]:
# === Generate Volcano Plots for All Comparisons ===

PLOT_VOLCANOS = False  # Set to True after DESeq2 is complete

if PLOT_VOLCANOS:
    # Find all results files
    results_files = list(DIRS['deseq2'].glob('group*_*_vs_*.csv'))
    results_files += list(DIRS['deseq2'].glob('group*_circadian_LRT.csv'))
    
    print(f"Found {len(results_files)} results files\n")
    
    for results_file in results_files:
        # Extract comparison name from filename
        name = results_file.stem
        title = name.replace('_', ' ').replace('vs', 'vs.')
        
        output_path = DIRS['figures'] / 'volcano_plots' / f"{name}.html"
        
        fig = create_volcano_plot(
            results_file, 
            title=title,
            output_path=output_path
        )
        # Display first one as example
        if results_file == results_files[0]:
            fig.show()
    
    print(f"\nAll volcano plots saved to: {DIRS['figures']}/volcano_plots/")
else:
    print("Skipping volcano plots (set PLOT_VOLCANOS = True after DESeq2 is complete)")

### 4.3 Expression Heatmaps

In [ ]:
# === Heatmap Function ===

def create_expression_heatmap(counts_df, gene_list, sample_ids, metadata, 
                               title="Expression Heatmap", output_path=None):
    """
    Create a clustered heatmap of gene expression.
    
    Parameters:
    -----------
    counts_df : pd.DataFrame
        Normalized counts matrix (genes × samples)
    gene_list : list
        Genes to include
    sample_ids : list
        Samples to include
    metadata : pd.DataFrame
        Sample metadata
    title : str
        Plot title
    output_path : Path, optional
        Path to save figure
    """
    # Subset data
    data = counts_df.loc[gene_list, sample_ids]
    
    # Log transform and z-score
    log_data = np.log2(data + 1)
    z_data = (log_data.T - log_data.mean(axis=1)) / log_data.std(axis=1)
    z_data = z_data.T
    
    # Get condition labels
    sample_meta = metadata[metadata['sample_id'].isin(sample_ids)].set_index('sample_id')
    conditions = sample_meta.loc[sample_ids, 'condition'].tolist()
    
    # Create heatmap
    fig = px.imshow(
        z_data,
        labels=dict(x="Sample", y="Gene", color="Z-score"),
        x=sample_ids,
        y=gene_list,
        color_continuous_scale='RdBu_r',
        zmin=-2, zmax=2,
        title=title,
        aspect='auto'
    )
    
    fig.update_layout(height=max(400, len(gene_list) * 15), width=900)
    
    if output_path:
        fig.write_html(output_path)
        print(f"Saved: {output_path}")
    
    return fig


print("Heatmap function defined.")

In [ ]:
# === Top DE Genes Heatmap ===

PLOT_HEATMAPS = False  # Set to True after DESeq2 is complete

if PLOT_HEATMAPS:
    # Load normalized counts
    norm_counts = pd.read_csv(DIRS['deseq2'] / 'counts_matrix.csv', index_col=0)
    
    # Example: Top DE genes from Group 2 High_NaCl comparison
    results_file = DIRS['deseq2'] / 'group2_High_NaCl_9pct_vs_Control.csv'
    
    if results_file.exists():
        results = pd.read_csv(results_file)
        
        # Get top 50 DE genes by adjusted p-value
        de_genes = results[
            (results['padj'] < 0.05) & 
            (abs(results['log2FoldChange']) > 1)
        ].nsmallest(50, 'padj')['gene_id'].tolist()
        
        # Filter to genes in counts matrix
        de_genes = [g for g in de_genes if g in norm_counts.index]
        
        # Get Group 2 samples
        group2_samples = metadata[metadata['group'] == 2]['sample_id'].tolist()
        
        if de_genes:
            fig = create_expression_heatmap(
                norm_counts, de_genes, group2_samples, metadata,
                title=f"Top {len(de_genes)} DE Genes: High NaCl vs Control",
                output_path=DIRS['figures'] / 'heatmaps' / 'top_de_high_nacl.html'
            )
            fig.show()
        else:
            print("No significant DE genes found.")
    else:
        print(f"Results file not found: {results_file}")
else:
    print("Skipping heatmaps (set PLOT_HEATMAPS = True after DESeq2 is complete)")

---
## Summary and Next Steps

In [ ]:
# === Pipeline Summary ===

print("=" * 60)
print("RNA-seq Pipeline Summary")
print("=" * 60)

# Check what outputs exist
def count_files(directory, pattern):
    """Count files matching pattern in directory."""
    if not directory.exists():
        return 0
    return len(list(directory.glob(pattern)))

print("\n[Data Inputs]")
print(f"  Raw FASTQ files: {count_files(DIRS['data'], '*.fastq.gz')}")

print("\n[Pre-processing]")
print(f"  FastQC raw reports: {count_files(DIRS['qc'] / 'fastqc_raw', '*_fastqc.html')}")
print(f"  Trimmed files: {count_files(DIRS['trimmed'], '*.fastq.gz')}")
print(f"  Merged files: {count_files(DIRS['merged'], '*.fastq.gz')}")
print(f"  rRNA-filtered files: {count_files(DIRS['rrna_filtered'] / 'non_rRNA', '*.fq.gz')}")

print("\n[Quantification]")
print(f"  Salmon quant directories: {count_files(DIRS['salmon'] / 'quants', '*')}")

print("\n[Alignment]")
print(f"  BAM files: {count_files(DIRS['alignment'] / 'bam', '*.bam')}")
print(f"  bigWig files: {count_files(DIRS['alignment'] / 'bigwig', '*.bw')}")

print("\n[DE Analysis]")
print(f"  Results files: {count_files(DIRS['deseq2'], '*.csv')}")

print("\n[Figures]")
print(f"  QC plots: {count_files(DIRS['figures'] / 'qc_plots', '*.html')}")
print(f"  Volcano plots: {count_files(DIRS['figures'] / 'volcano_plots', '*.html')}")
print(f"  Heatmaps: {count_files(DIRS['figures'] / 'heatmaps', '*.html')}")

print("\n" + "=" * 60)

## Next Steps

### To run the full pipeline:

1. **Activate conda environment**:
   ```bash
   conda activate pcc11901_rnaseq
   ```

2. **Set the RUN_* flags to True** for each step you want to execute

3. **Execute cells in order** (or run shell scripts separately for compute-intensive steps)

### Recommended execution order:

| Step | Flag | Notes |
|------|------|-------|
| FastQC raw | `RUN_FASTQC_RAW` | ~30 min for 240 files |
| fastp trimming | `RUN_FASTP` | ~2-3 hours |
| Lane merging | `RUN_MERGE` | Fast (~10 min) |
| SortMeRNA | `RUN_SORTMERNA` | SLOW (~1 hour/sample, memory-intensive) |
| FastQC trimmed | `RUN_FASTQC_TRIMMED` | ~15 min |
| MultiQC | `RUN_MULTIQC` | Fast |
| gffread | `RUN_GFFREAD` | Fast |
| Salmon index | `RUN_SALMON_INDEX` | ~5-10 min |
| Salmon quant | `RUN_SALMON_QUANT` | ~30 min total |
| Bowtie2 index | `RUN_BOWTIE2_INDEX` | ~5 min |
| Bowtie2 align | `RUN_BOWTIE2_ALIGN` | ~2-3 hours |
| bigWig | `RUN_BIGWIG` | ~30 min |
| Load counts | `LOAD_COUNTS` | Fast |
| DESeq2 Group 1 | `RUN_DESEQ2_GROUP1` | Fast |
| DESeq2 Group 2 | `RUN_DESEQ2_GROUP2` | Fast |
| DESeq2 Group 3 | `RUN_DESEQ2_GROUP3` | Fast |
| Visualization | `PLOT_*` | Fast |

### Future analyses (not implemented):

- Functional enrichment (GO, KEGG) via eggNOG-mapper + clusterProfiler
- Gene set enrichment analysis (GSEA)
- Co-expression network analysis
- Pathway visualization
